In [1]:
import numpy as np
import cv2
import open3d as o3d
import time

# 读取深度图像
depth_img = cv2.imread("../img/1_depth.png", cv2.IMREAD_GRAYSCALE)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [18]:
w, h = depth_img.shape
# 相机内参矩阵
k = np.array([[535.4, 0.0, 320.0],
              [0.0, 539.2, 240.0],
              [0.0, 0.0, 1.0]])

# 每次都一样，避免重复计算
i, j = np.indices((w, h))
static_array = np.stack((i, j, np.ones((w, h))), axis=2).reshape(-1, 3)
# 获取相机内参的逆矩阵
k_inv = np.linalg.inv(k)

# 每行乘逆矩阵
point_array = k_inv@static_array.T
    
def depth_to_point_cloud_v3(depth,_point_array):
    #ssad
    depth_flat = depth.astype(np.float32).reshape(-1,1)
    # 每行乘深度值
    point_cloud = depth_flat * _point_array.T
    return point_cloud

In [3]:
# 说明大多开销在    static_array = np.array([np.array([i,j, 1]) for i in range(w) for j in range(h)])

# 加速代码
i, j = np.indices((w, h))
_static_array = np.stack((i, j, np.ones((w, h))), axis=2).reshape(-1, 3) #0.004s

# 原理说明：
# 第一个矩阵是行数阵，第二个是列数阵，第三个是全一阵，把三个矩阵堆叠，再reshape即可；通过这种方法避免循环太慢
np.indices((w, h))

array([[[  0,   0,   0, ...,   0,   0,   0],
        [  1,   1,   1, ...,   1,   1,   1],
        [  2,   2,   2, ...,   2,   2,   2],
        ...,
        [477, 477, 477, ..., 477, 477, 477],
        [478, 478, 478, ..., 478, 478, 478],
        [479, 479, 479, ..., 479, 479, 479]],

       [[  0,   1,   2, ..., 637, 638, 639],
        [  0,   1,   2, ..., 637, 638, 639],
        [  0,   1,   2, ..., 637, 638, 639],
        ...,
        [  0,   1,   2, ..., 637, 638, 639],
        [  0,   1,   2, ..., 637, 638, 639],
        [  0,   1,   2, ..., 637, 638, 639]]])

In [14]:
# 十次求平均
elapsed_time_list = []
for i in range(10):# 
    start_time = time.time()  # 记录开始时间

    # 将深度图转换为点云
    pc = depth_to_point_cloud_v3(depth_img,point_array)

    end_time = time.time()  # 记录结束时间

    elapsed_time = end_time - start_time  # 计算耗时

    elapsed_time_list.append(elapsed_time)

average_time = sum(elapsed_time_list) / len(elapsed_time_list)

print(f"Time elapsed average 10 times:{average_time:.4f} seconds.")

Time elapsed average 10 times:0.0027 seconds.


In [20]:
import cProfile
cProfile.run('depth_to_point_cloud_v3(depth_img,point_array)')

         6 function calls in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.003    0.003    0.003    0.003 3098973681.py:16(depth_to_point_cloud_v3)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        1    0.000    0.000    0.004    0.004 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'astype' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'reshape' of 'numpy.ndarray' objects}




In [9]:
# 可视化点云

# 点云数据存储在名为point_cloud的numpy数组中
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(pc)

# 创建一个可视化窗口
vis = o3d.visualization.Visualizer()
vis.create_window()

# 将点云添加到可视化窗口中
vis.add_geometry(cloud)

# 设置相机位置，没调所以可有可无
vis.get_view_control().set_front([0, 0, -1])
vis.get_view_control().set_lookat([0, 0, 0])
vis.get_view_control().set_up([0, -1, 0])

# 开始显示点云
vis.run()

# 关闭显示窗口
vis.destroy_window()

0.0027s! 超越open3d!定深度图尺寸下、定内参矩阵、不考虑畸变的情况加速300倍